In [10]:
import sqlite3
import os
import pandas as pd

### Reading current db with metadata

In [11]:
pdf_db_path = "../app_storage/metadata/sqlite-poc.db"
db = sqlite3.connect(pdf_db_path)
df = pd.read_sql_query("SELECT * FROM pdf_metadata;", db)
df.head()

,id,file_name,file_path,title,author,creation_date,subject
0,1,Bayesian_non_parametric.pdf,app_storage/pdfs\Bayesian_non_parametric.pdf,Untitled,Unknown,D:20100128111432Z,N/A
1,2,part3.MR_Yarn François.pdf,app_storage/pdfs\part3.MR_Yarn François.pdf,part3.MR_Yarn,Unknown,D:20231215074950Z,N/A
2,3,Docker CLI Cheatsheet.pdf,app_storage/pdfs\Docker CLI Cheatsheet.pdf,Untitled,Unknown,D:20220606130303-04'00',N/A
3,4,test1.pdf,app_storage/pdfs/test1.pdf,test1,Unknown,D:20241005153153Z00'00',N/A
4,5,test1.pdf,app_storage/pdfs/test1.pdf,test1,Unknown,D:20241005153153Z00'00',N/A


### Demo - sqlite to store and retrieve embeddings

https://github.com/asg017/sqlite-vec/blob/main/examples/python-recipes/openai-sample.py

In [12]:
import sqlite_vec

from typing import List
import struct


def serialize_f32(vector: List[float]) -> bytes:
    """serializes a list of floats into a compact "raw bytes" format"""
    return struct.pack("%sf" % len(vector), *vector)


db.enable_load_extension(True)
sqlite_vec.load(db)
db.enable_load_extension(False)

items = [
    (1, [0.1, 0.1, 0.1, 0.1]),
    (2, [0.2, 0.2, 0.2, 0.2]),
    (3, [0.3, 0.3, 0.3, 0.3]),
    (4, [0.4, 0.4, 0.4, 0.4]),
    (5, [0.5, 0.5, 0.5, 0.5]),
]
query = [0.3, 0.3, 0.3, 0.3]

db.execute("DROP TABLE IF EXISTS vec_items")
db.execute("CREATE VIRTUAL TABLE vec_items USING vec0(embedding float[4])")

with db:
    for item in items:
        db.execute(
            "INSERT INTO vec_items(rowid, embedding) VALUES (?, ?)",
            [item[0], serialize_f32(item[1])],
        )

rows = db.execute(
    """
      SELECT
        rowid,
        distance
      FROM vec_items
      WHERE embedding MATCH ?
      ORDER BY distance
      LIMIT 3
    """,
    [serialize_f32(query)],
).fetchall()

print(f"The closer vectors to the query are: {rows}")


The closer vectors to the query are: [(3, 0.0), (4, 0.19999998807907104), (2, 0.20000001788139343)]


###  Storing embeddings in practice 

In [15]:
# Store the embeddings in a table

embeddings = [
    (1, [0.1, 0.1, 0.1, 0.1], 1),
    (2, [0.2, 0.2, 0.2, 0.2], 1),
    (3, [0.3, 0.3, 0.3, 0.3], 1),
    (4, [0.4, 0.4, 0.4, 0.4], 1),
]

db.execute("DROP TABLE IF EXISTS chunk_embeddings")
db.execute("""
           CREATE VIRTUAL TABLE chunk_embeddings USING vec0(
                    chunk_id INTEGER PRIMARY KEY,
                    embedding float[4],
                    );
           """)  ## Note: 4 is for the number of dimensions in the embedding
with db:
    for embedding in embeddings:
        db.execute(
            "INSERT INTO chunk_embeddings(chunk_id, embedding) VALUES (?, ?)",
            [embedding[0], serialize_f32(embedding[1])],
        )
pd.read_sql_query("SELECT * FROM chunk_embeddings;", db)

,chunk_id,embedding
0,1,b'\xcd\xcc\xcc=\xcd\xcc\xcc=\xcd\xcc\xcc=\xcd\...
1,2,b'\xcd\xccL>\xcd\xccL>\xcd\xccL>\xcd\xccL>'
2,3,b'\x9a\x99\x99>\x9a\x99\x99>\x9a\x99\x99>\x9a\...
3,4,b'\xcd\xcc\xcc>\xcd\xcc\xcc>\xcd\xcc\xcc>\xcd\...


###  Store the chunks in another table (normal sqlite)

In [6]:

sentences = [
    (1, "Capri-Sun is a brand of juice concentrate–based drinks.", 1),
    (2, "George V was King of the United Kingdom .", 1),
    (3, "Alaqua Cox is an actress.", 1),
    (4, "Shohei Ohtani is a Japanese professional.", 1),
]

db.execute("DROP TABLE IF EXISTS chunk_content")
db.execute(
    """
        CREATE TABLE chunk_content(
          chunk_id INTEGER PRIMARY KEY,
          sentence TEXT,
          document_id INTEGER
        );
    """
)

with db:
    for sentence in sentences:
        db.execute(
            "INSERT INTO chunk_content(chunk_id, sentence, document_id) VALUES (?, ?, ?)",
            sentence,
        )
pd.read_sql_query("SELECT * FROM chunk_content;", db)

,chunk_id,sentence,document_id
0,1,Capri-Sun is a brand of juice concentrate–base...,1
1,2,George V was King of the United Kingdom .,1
2,3,Alaqua Cox is an actress.,1
3,4,Shohei Ohtani is a Japanese professional.,1


We now have 3 tables in the database: one for the pdfs and their metadata, one for the chunks and their embeddings, and one for the chunks and their content.

### Reconciliating the three tables

In [7]:
pd.read_sql_query("""
           SELECT * 
           FROM chunk_content 
           JOIN chunk_embeddings ON chunk_content.chunk_id = chunk_embeddings.chunk_id
           LEFT JOIN pdf_metadata ON chunk_content.document_id = pdf_metadata.id
           """
           , db)

,chunk_id,sentence,document_id,chunk_id,embedding,id,file_name,file_path,title,author,creation_date,subject
0,1,Capri-Sun is a brand of juice concentrate–base...,1,1,b'\xcd\xcc\xcc=\xcd\xcc\xcc=\xcd\xcc\xcc=\xcd\...,1,Bayesian_non_parametric.pdf,app_storage/pdfs\Bayesian_non_parametric.pdf,Untitled,Unknown,D:20100128111432Z,N/A
1,2,George V was King of the United Kingdom .,1,2,b'\xcd\xccL>\xcd\xccL>\xcd\xccL>\xcd\xccL>',1,Bayesian_non_parametric.pdf,app_storage/pdfs\Bayesian_non_parametric.pdf,Untitled,Unknown,D:20100128111432Z,N/A
2,3,Alaqua Cox is an actress.,1,3,b'\x9a\x99\x99>\x9a\x99\x99>\x9a\x99\x99>\x9a\...,1,Bayesian_non_parametric.pdf,app_storage/pdfs\Bayesian_non_parametric.pdf,Untitled,Unknown,D:20100128111432Z,N/A
3,4,Shohei Ohtani is a Japanese professional.,1,4,b'\xcd\xcc\xcc>\xcd\xcc\xcc>\xcd\xcc\xcc>\xcd\...,1,Bayesian_non_parametric.pdf,app_storage/pdfs\Bayesian_non_parametric.pdf,Untitled,Unknown,D:20100128111432Z,N/A


In [8]:
db.close()